In [7]:
%matplotlib inline
from pyspark.mllib.tree import RandomForest
import xgboost as xgb
from pyspark.mllib.util import MLUtils
from pyspark.mllib.tree import GradientBoostedTrees
import matplotlib
import numpy as np
from operator import add
import matplotlib.pyplot as plt
from pyspark.mllib.regression import LabeledPoint
l=sc.textFile("./al",1000).filter(lambda x:"-" in x).map(lambda x:list(map(int,x.split(",")[1:])))
l.first()

[28, 0, 1130, 1710, 2209, 0]

In [11]:
(train, test) = l.randomSplit([0.7, 0.3],1)
t=train
data=np.array(t.map(lambda x:x[:-1]).collect())
label=np.array(t.map(lambda x:float(x[-1])).collect())
xgtrain = xgb.DMatrix( data, label=label)
t=test
data=np.array(t.map(lambda x:x[:-1]).collect())
label=np.array(t.map(lambda x:float(x[-1])).collect())
xgtest = xgb.DMatrix( data, label=label)

In [ ]:
from pyspark.mllib.classification import LogisticRegressionWithLBFGS, LogisticRegressionModel,LogisticRegressionWithSGD
model2 = LogisticRegressionWithLBFGS.train(train,5,regParam =0.001,regType='l2')
        # Evaluating the model on training data
labelsAndPreds2 = test.map(lambda p: (p.label, model2.predict(p.features)))
labelsAndPreds2.filter(lambda x:x[0]==x[1]).count()

In [ ]:
t=train
data=np.array(t.map(lambda x:x.features).collect())
label=np.array(t.map(lambda x:x.label).collect())
xgtrain = xgb.DMatrix( data, label=label)
t=test
data=np.array(t.map(lambda x:x.features).collect())
label=np.array(t.map(lambda x:x.label).collect())
xgtest = xgb.DMatrix( data, label=label)

In [17]:
# t=train
# data=np.array(t.map(lambda x:x.features).collect())
# label=np.array(t.map(lambda x:x.label).collect())
# xgtrain = xgb.DMatrix( data, label=label)
# t=test
# data=np.array(t.map(lambda x:x.features).collect())
# label=np.array(t.map(lambda x:x.label).collect())
# xgtest = xgb.DMatrix( data, label=label)
params={
'booster':'gbtree',
# 这里手写数字是0-9，是一个多类的问题，因此采用了multisoft多分类器，
'objective': 'binary:logistic', 
'subsample':1,
'gamma':0.5,  # 在树的叶子节点下一个分区的最小损失，越大算法模型越保守 。[0:]
'max_depth':2, # 构建树的深度 [1:]
#'lambda':450,  # L2 正则项权重
'colsample_bytree':0.8, # 构建树树时的采样比率 (0:1]
'min_child_weight':9,
'silent':1 ,
'eta': 0.1, # 如同学习率
'seed':710,
'nthread':4,# cpu 线程数,根据自己U的个数适当调整
}
plst = list(params.items())

# return 训练和验证的错误率
# watchlist = [(xgtrain, 'train')]
watchlist = [(xgtrain, 'train'),(xgtest, 'val')]
num_rounds=3000
# training model 
# early_stopping_rounds 当设置的迭代次数较大时，early_stopping_rounds 可在一定的迭代次数内准确率没有提升就停止训练
model = xgb.train(plst, xgtrain, num_rounds, watchlist,early_stopping_rounds=50)
#model.save_model('./model/xgb.model') # 用于存储训练出的模型
preds = model.predict(xgtest,ntree_limit=model.best_iteration)

[0]	train-error:0.377049	val-error:0.421053
Multiple eval metrics have been passed: 'val-error' will be used for early stopping.

Will train until val-error hasn't improved in 50 rounds.
[1]	train-error:0.377049	val-error:0.421053
[2]	train-error:0.377049	val-error:0.421053
[3]	train-error:0.377049	val-error:0.421053
[4]	train-error:0.377049	val-error:0.421053
[5]	train-error:0.377049	val-error:0.421053
[6]	train-error:0.377049	val-error:0.421053
[7]	train-error:0.377049	val-error:0.421053
[8]	train-error:0.377049	val-error:0.421053
[9]	train-error:0.377049	val-error:0.421053
[10]	train-error:0.377049	val-error:0.421053
[11]	train-error:0.377049	val-error:0.421053
[12]	train-error:0.377049	val-error:0.421053
[13]	train-error:0.377049	val-error:0.421053
[14]	train-error:0.377049	val-error:0.421053
[15]	train-error:0.377049	val-error:0.421053
[16]	train-error:0.377049	val-error:0.421053
[17]	train-error:0.377049	val-error:0.421053
[18]	train-error:0.377049	val-error:0.421053
[19]	train-e